# Reaction Manipulations

The division of the ME-model into MEReaction and ProcessData classes allows the user to essentially have access to the entire database of information used to construct the model. The following will show how this can be leveraged to easily query, edit and update aspects of the model reactions.

In [1]:
import cobrame
import cPickle
from collections import defaultdict

In [2]:
with open('/home/sbrg-cjlloyd/ecolime/prototype_notebooks/prototype_67.pickle') as f:
    me = cPickle.load(f)

## Metabolic Reactions

In [3]:
print('number of metabolic reactions = %i' % 
      len([r.id for r in me.reactions if type(r) == cobrame.MetabolicReaction]))

number of metabolic reactions = 5285


Through a MetabolicReaction, the user has direct access to the StoichiometricData, ComplexData and keff used to construct the reaction.

In [4]:
rxn = me.reactions.get_by_id('E4PD_FWD_GAPDH-A-CPLX')
stoich_data = rxn.stoichiometric_data
complex_data = rxn.complex_data

print(rxn.reaction + '\n')
print('This reactions is formed using the StoichiometricData (%s) and ComplexData (%s) with keff = %.2f' % 
      (stoich_data.id, complex_data.id, rxn.keff))

3.20942472231275e-6*mu GAPDH-A-CPLX + e4p_c + h2o_c + nad_c --> 4per_c + 2.0 h_c + nadh_c

This reactions is formed using the StoichiometricData (E4PD) and ComplexData (GAPDH-A-CPLX) with keff = 86.55


As a best practice, the ComplexData and StoichiometricData themselves should not be changed. If these need changed then a new MetabolicReaction should be created. 

### Edit keffs
Further, the keff is an attribute of the MetabolicReaction itself and not of the ComplexData changing the ComplexData will not affect the form of the MetabolicReaction.



In [5]:
print('keff = %d: \n\t%s' % (rxn.keff, rxn.reaction))
rxn.keff = 65.
rxn.update()
print('keff = %d: \n\t%s' % (rxn.keff, rxn.reaction))

keff = 86: 
	3.20942472231275e-6*mu GAPDH-A-CPLX + e4p_c + h2o_c + nad_c --> 4per_c + 2.0 h_c + nadh_c
keff = 65: 
	4.27350427350427e-6*mu GAPDH-A-CPLX + e4p_c + h2o_c + nad_c --> 4per_c + 2.0 h_c + nadh_c


### Edit stoichiometry

Aspects of the StoichiometricData, however, can be changed. This includes:
 - Reaction stoichiometry
 - Reaction upper & lower bounds
 
Currently the stoichiometry is:

In [6]:
stoich_data.stoichiometry

{'4per_c': 1.0,
 'e4p_c': -1.0,
 'h2o_c': -1.0,
 'h_c': 2.0,
 'nad_c': -1.0,
 'nadh_c': 1.0}

This can be updated to, for instance, translocate a hydrogen by performing the following

In [7]:
stoich_data.stoichiometry['h_c'] = -1
stoich_data.stoichiometry['h_p'] = 1
rxn.update()
print('%s: \n\t%s' % (rxn.id, rxn.reaction))

E4PD_FWD_GAPDH-A-CPLX: 
	4.27350427350427e-6*mu GAPDH-A-CPLX + e4p_c + h2o_c + h_c + nad_c --> 4per_c + h_p + nadh_c


This change can be usd to update both the forward and reverse reaction

In [8]:
rxn_rev = me.reactions.get_by_id(rxn.id.replace('FWD', 'REV'))
rxn_rev.update()
print('%s: \n\t%s' % (rxn_rev.id, rxn_rev.reaction))

E4PD_REV_GAPDH-A-CPLX: 
	4per_c + 3.20942472231275e-6*mu GAPDH-A-CPLX + h_p + nadh_c --> e4p_c + h2o_c + h_c + nad_c


A simpler approach is to updated the parent reactions for StoichiometricData. This will update any instances of the reaction catalyzed by an isozyme.

In [9]:
stoich_data.stoichiometry['h_c'] = -2
stoich_data.stoichiometry['h_p'] = 2
for r in stoich_data.parent_reactions:
    r.update()
    print('%s: \n\t%s' % (r.id, r.reaction))

E4PD_FWD_ERYTH4PDEHYDROG-CPLX: 
	0.0135143204065698*mu ERYTH4PDEHYDROG-CPLX + e4p_c + h2o_c + 2.0 h_c + nad_c --> 4per_c + 2.0 h_p + nadh_c
E4PD_FWD_GAPDH-A-CPLX: 
	4.27350427350427e-6*mu GAPDH-A-CPLX + e4p_c + h2o_c + 2.0 h_c + nad_c --> 4per_c + 2.0 h_p + nadh_c
E4PD_REV_ERYTH4PDEHYDROG-CPLX: 
	4per_c + 3.09490345954754e-6*mu ERYTH4PDEHYDROG-CPLX + 2.0 h_p + nadh_c --> e4p_c + h2o_c + 2.0 h_c + nad_c
E4PD_REV_GAPDH-A-CPLX: 
	4per_c + 3.20942472231275e-6*mu GAPDH-A-CPLX + 2.0 h_p + nadh_c --> e4p_c + h2o_c + 2.0 h_c + nad_c


### Edit upper and lower reaction bounds
The upper and lower bounds can be edited through the stoichiometric data and updated to the metabolic reaction

**Important: do not change the upper and lower bounds of a MetabolicReaction directly. If this is done than the change will be overwritten when the update function is ran (shown below)**

In [10]:
rxn.lower_bound = -1000
print('Lower bound = %d' %rxn.lower_bound)
rxn.update()
print('Lower bound = %d' %rxn.lower_bound)

Lower bound = -1000
Lower bound = 0


Editing the reaction bounds of the StoichiometricData, however, will edit the bounds of the forward and reverse reaction, as well as any instances of the reaction catalyzed by isozymes 

In [11]:
stoich_data.lower_bound
stoich_data.lower_bound = 0.
print('Upper Bounds\n--------------------------------------')
for r in stoich_data.parent_reactions:
    direction = 'Forward' if r.reverse is False else 'Reverse'
    print('%s Before Update \n\t%s: %s' % (direction, r.id, r.upper_bound))
    r.update()
    print('%s After Update \n\t%s: %s' % (direction, r.id, r.upper_bound))

Upper Bounds
--------------------------------------
Forward Before Update 
	E4PD_FWD_ERYTH4PDEHYDROG-CPLX: 1000.0
Forward After Update 
	E4PD_FWD_ERYTH4PDEHYDROG-CPLX: 1000.0
Forward Before Update 
	E4PD_FWD_GAPDH-A-CPLX: 1000.0
Forward After Update 
	E4PD_FWD_GAPDH-A-CPLX: 1000.0
Reverse Before Update 
	E4PD_REV_ERYTH4PDEHYDROG-CPLX: 1000.0
Reverse After Update 
	E4PD_REV_ERYTH4PDEHYDROG-CPLX: 0
Reverse Before Update 
	E4PD_REV_GAPDH-A-CPLX: 1000.0
Reverse After Update 
	E4PD_REV_GAPDH-A-CPLX: 0


## Transcription Reactions

In [12]:
print('number of transcription reactions = %i' % 
      len([r.id for r in me.reactions if type(r) == cobrame.TranscriptionReaction]))
print('number of transcription data objects = %i' %  len(me.transcription_data))

number of transcription reactions = 1453
number of transcription data objects = 1453


Transciption occurs via operons contained within the organisms genome or transcription unit (TU). This means that often, a transcribed region will code for multiple RNAs. The E. coli ME-model has 4 possible RNA types that can be transcribed:
 - mRNA
 - tRNA
 - rRNA
 - ncRNA (noncoding RNA)

mRNAs can then translated directly from the full transcribed TU, while rRNA, tRNA and ncRNA are spliced out of the TU by endonucleases. In these cases, in order to know which bases need excized, the RNA metabolites (TranscribedGene) themselves have to store information such as:
 - **DNA strand, left and right genome position** to identify which TU the RNA is a part of
 - **RNA type** to determine whether it needs excised from the TU
 - **nucleotide sequence** to determine bases that do/do not need excised if not mRNA and the RNA mass for biomass constraint

An example of a TranscribedGene's attributes is shown below

In [13]:
for key, value in me.metabolites.RNA_b3201.__dict__.items():
    if not key.startswith('_') and value:
        if type(value) is str and len(value) > 50:
            value = value[:50] + '...'
        print key, value

right_pos 3342691
formula C6890H8542N2720O5817P726
RNA_type mRNA
left_pos 3341965
nucleotide_sequence ATGGCAACATTAACTGCAAAGAACCTTGCAAAAGCCTATAAAGGCCGTCG...
id RNA_b3201
strand +


Each TranscriptionReaction in a COBRAme ME-model is associated with exactly one TranscriptionData which includes everything necessary to define a reaction. This includes:
 - **subreactions** To handle enzymatic processes not performed by RNA polymerase
 - **RNA Polymerase** Different RNA polymerase metabolite for different sigma factors
 - **RNA Products** TUs often contain more than one RNA in sequence
 - **Nucleotide sequence**

The TranscriptionData for TU containing the gene above is shown below

In [14]:
rxn = me.reactions.transcription_TU_8398_from_RPOE_MONOMER
data = rxn.transcription_data
for key, value in data.__dict__.items():
    if not key.startswith('_') and value:
        if type(value) is str and len(value) > 50:
            value = value[:50] + '...'
        print key, value

subreactions defaultdict(<type 'int'>, {'Transcription_normal_rho_dependent': 1})
rho_dependent True
RNA_polymerase RNAPE-CPLX
nucleotide_sequence ACAAACTCAGCCTTAATCTTGTGCTTGCCAGCTCACTTCTGGCCGCCAGC...
RNA_products set(['RNA_b3201', 'RNA_b3202'])
id TU_8398_from_RPOE_MONOMER


This reaction currently uses a subreaction called *Transcription_normal_rho_dependent* to account for the elongation factors etc. associated with transcription. This TU also requires a rho factor to terminate transcription. These complexes can be removed from the reaction by running the following

In [15]:
print('with subreactions: ' + rxn.reaction)
data.subreactions = {}
for r in data.parent_reactions:
    r.update()
print('\nwithout subreactions: ' + rxn.reaction)

with subreactions: 4.27350427350427e-6*mu GreA_mono + 4.27350427350427e-6*mu GreB_mono + 4.27350427350427e-6*mu Mfd_mono_mod_1:mg2 + 4.27350427350427e-6*mu NusA_mono + 4.27350427350427e-6*mu NusG_mono + 0.0186826474945534*mu + 0.00731425649411765 RNAPE-CPLX + 4.27350427350427e-6*mu Rho_hexa_mod_3:mg2 + 4.27350427350427e-6*mu RpoZ_mono_mod_1:mg2 + 1020 atp_c + 1181 ctp_c + 1190 gtp_c + 4577 h2o_c + 1186 utp_c --> RNA_b3201 + RNA_b3202 + 3 adp_c + 4577 h_c + 691.702633 mRNA_biomass + 3 pi_c + 4574 ppi_c

without subreactions: 0.0186826474945534*mu + 0.00731425649411765 RNAPE-CPLX + 1017 atp_c + 1181 ctp_c + 1190 gtp_c + 4574 h2o_c + 1186 utp_c --> RNA_b3201 + RNA_b3202 + 4574 h_c + 691.702633 mRNA_biomass + 4574 ppi_c


This poses a problem where, if RNA_b3201 and RNA_b3202 are not required in equal amounts, the model will become infeasible. To accound for this, all RNAs have a demand reaction associated with them. *mRNA_biomass* is consumed for each demand reaction with a coefficient equal to the molecular weight of each RNA (in kDa). This prevents the model from overproducing RNA to increase biomass production, and therefore growth rate, in some instances. More on the implications of the *biomass* constraint can be found in **ME-Model Fundamentals**

In [16]:
for rna in data.RNA_products:
    r = me.reactions.get_by_id('DM_' + rna)
    print('%s: %s' % (r.id, r.reaction))

DM_RNA_b3201: RNA_b3201 + 232.671391 mRNA_biomass --> 
DM_RNA_b3202: RNA_b3202 + 459.031242 mRNA_biomass --> 


As is, this reaction produces two mRNAs so no nucleotides are excised. If one or both is changed to a stable RNA (rRNA, tRNA or ncRNA) bases will be excised.

In [17]:
me.metabolites.RNA_b3202.RNA_type = 'rRNA'
for r in data.parent_reactions:
    r.update()
    print(r.reaction)

0.0186826474945534*mu + 0.00731425649411765 RNAPE-CPLX + 1017 atp_c + 1181 ctp_c + 1190 gtp_c + 4574 h2o_c + 1186 utp_c --> RNA_b3201 + RNA_b3202 + 557 amp_c + 605 cmp_c + 613 gmp_c + 4574 h_c + 232.671391 mRNA_biomass + 4575 ppi_c + 459.031242 rRNA_biomass + 639 ump_c


Changing RNA_b3202 to an rRNA and updating the transcription reaction causes both of the RNAs to now be excised from the TU, as indicated by the nucleotide monophosphates that appear in the products. This is not a complete picture because this process is catalyzes by an endonuclease, whose activity can be incorporated as ModificationData. Updating the reaction after adding these processes incorporates 

In [20]:
data.modifications['rRNA_containing_excision'] = len(data.RNA_products) * 2
data.modifications['RNA_degradation_machine'] = len(data.RNA_products) * 2
data.modifications['RNA_degradation_atp_requirement'] = sum(data.excised_bases.values())
for r in data.parent_reactions:
    r.update()
    print(r.reaction)

0.0186826474945534*mu + 0.00731425649411765 RNAPE-CPLX + 1.70940170940171e-5*mu RNA_degradosome + 1620.5 atp_c + 1181 ctp_c + 1190 gtp_c + 7595.5 h2o_c + 1.70940170940171e-5*mu rRNA_containing_excision_machinery + 1186 utp_c --> RNA_b3201 + RNA_b3202 + 603.5 adp_c + 557 amp_c + 605 cmp_c + 613 gmp_c + 7595.5 h_c + 232.671391 mRNA_biomass + 603.5 pi_c + 4575 ppi_c + 459.031242 rRNA_biomass + 639 ump_c


## Translation Reactions